In [2]:
%load_ext tensorboard

In [1]:
import os
import h5py
import random
import datetime
import itertools
import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
import albumentations as A
import tensorflow_hub as hub
import matplotlib.pylab as plt

from pathlib import PurePath
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from google.colab.patches import cv2_imshow as cv_imshow

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Copy dataset to Google Colab storage to improve I/O speed.

In [6]:
!cp '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/dataset_128.h5' .

#### Constants

In [4]:
AUTOTUNE = tf.data.AUTOTUNE
FINE_TUNE = True
SEED = 67

BATCH_SIZE = 128
BUFFER_SIZE = 8

TRAINING_SPLIT = 0.7
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.1

EPOCHS = 15

In [33]:
dataset_dir = './dataset_128.h5'
checkpoint_path = "checkpoint_mobilenetv2.ckpt"
models_dir = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/models'
checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/checkpoints"
train_filepath = '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/dataset/train.csv' 

logs_dir = "/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/logs"

## Model

In [7]:
# Image size should be compatible with backbone model input
IMAGE_SIZE = (128, 128)

model_name = "custom_mobilenet"
model_version = "1_2"

def Model(num_classes):
    x = inputs = tf.keras.layers.Input(IMAGE_SIZE + (3,), name = "input")

    # Data Augmentation Layer
    x = tf.keras.layers.RandomFlip(mode = "horizontal_and_vertical", seed = SEED)(x)
    x = tf.keras.layers.RandomRotation(factor = (-0.4, 0.4), fill_mode = "constant", interpolation = "bilinear", seed = SEED, fill_value = 0.0)(x)
    x = tf.keras.layers.RandomContrast(factor = 0.3, seed = SEED)(x)
    x = tf.keras.layers.Rescaling(1./255)(x)

    # Pre-Trained Model Backbone
    x = hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_128/feature_vector/5", trainable = FINE_TUNE)(x)

    # Classification Layer
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, kernel_regularizer = tf.keras.regularizers.l1_l2(l1 = 0, l2 = 2E-4), activation = tf.keras.activations.softmax)(x)

    model = tf.keras.Model(inputs, outputs, name = model_name)

    model.summary()

    return model


## Image Pre-Processing

In [8]:
hf = h5py.File(dataset_dir, 'r')

Further Image Augmentation using [albumentations](https://albumentations.ai/docs/).

In [9]:
random.seed(SEED) 

training_transform = A.Compose([
  A.CLAHE(p = 1),
  A.HueSaturationValue(hue_shift_limit = 20, sat_shift_limit = 50, val_shift_limit = 50, p = 0.5),
  A.ToFloat(max_value = 255.0),
])

In [10]:
def augmentation_function(img):
  data = {"image": img}
  
  img = training_transform(**data)["image"]

  img = tf.image.resize(img, IMAGE_SIZE)

  return img


Define function to read image from dataset

In [11]:
# Necessary wrapper function in order to get image path string from Tensor
def read_img_from_tensor(name):
  return tf.convert_to_tensor(hf[name.numpy()])

In [12]:
def read_img(name, label):  
  img = tf.py_function(read_img_from_tensor, [name], tf.uint8)

  # Using py_function means TensorFlow can no longer know the output Tensor shape so we need to explicitly set it
  img.set_shape(tf.TensorShape([None, None, 3]))

  # Uncomment for further image augmentation (besides model layers)
  # img = tf.numpy_function(func = augmentation_function, inp = [img], Tout = tf.float32)

  return (img, label)

## Dataset Generation

Define target label - either `hotel_id` or `chain`.

In [13]:
TARGET_LABEL = "hotel_id"

Read the training dataset.

In [14]:
df = pd.read_csv(train_filepath, parse_dates = ["timestamp"])

Remove image duplicates.

In [15]:
df = df.drop_duplicates(subset = ["image"], keep = "first")

Use a label enconder to tranform the Hotel IDs into values within the range of the number of classes.

In [17]:
le = LabelEncoder()

df[TARGET_LABEL] = le.fit_transform(df[TARGET_LABEL])

Split into train/validation/test dataframes.

In [19]:
train_df = df.sample(frac = TRAINING_SPLIT, random_state = SEED)
val_df = df.drop(train_df.index).sample(frac = VALIDATION_SPLIT / (1 - TRAINING_SPLIT), random_state = SEED)
test_df = df.drop(train_df.index).drop(val_df.index)

In [20]:
print(f"Training size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Training size: 68288
Validation size: 19511
Test size: 9755


Create tensors for labels and filenames.

In [21]:
label_depth = len(set(df[TARGET_LABEL].values))

train_filenames = tf.convert_to_tensor(train_df['image'].tolist(), dtype = tf.string)
val_filenames = tf.convert_to_tensor(val_df['image'].tolist(), dtype = tf.string)
test_filenames = tf.convert_to_tensor(test_df['image'].tolist(), dtype = tf.string)

# Use label representation
# train_label_files = tf.convert_to_tensor(np.array(train_df[TARGET_LABEL]), dtype = tf.int32)
# val_label_files = tf.convert_to_tensor(np.array(val_df[TARGET_LABEL]), dtype = tf.int32)
# test_label_files = tf.convert_to_tensor(np.array(test_df[TARGET_LABEL]), dtype = tf.int32)

# Use one hot vector representation
train_label_files = tf.one_hot(tf.convert_to_tensor(np.array(train_df[TARGET_LABEL]), dtype = tf.int32), depth = label_depth)
val_label_files = tf.one_hot(tf.convert_to_tensor(np.array(val_df[TARGET_LABEL]), dtype = tf.int32), depth = label_depth)
test_label_files = tf.one_hot(tf.convert_to_tensor(np.array(test_df[TARGET_LABEL]), dtype = tf.int32), depth = label_depth)

Create the datasets from the Tensors.

In [22]:
train_ds = tf.data.Dataset.from_tensor_slices((train_filenames, train_label_files)) \
                          .shuffle(len(train_filenames), seed = SEED) \
                          .map(read_img, num_parallel_calls = AUTOTUNE) \
                          .batch(BATCH_SIZE) \
                          .prefetch(buffer_size = BUFFER_SIZE) \

val_ds = tf.data.Dataset.from_tensor_slices((val_filenames, val_label_files)) \
                          .shuffle(len(val_filenames), seed = SEED) \
                          .map(read_img, num_parallel_calls = AUTOTUNE) \
                          .batch(BATCH_SIZE) \
                          .prefetch(buffer_size = BUFFER_SIZE) \

test_ds = tf.data.Dataset.from_tensor_slices((test_filenames, test_label_files)) \
                          .map(read_img, num_parallel_calls = AUTOTUNE) \
                          .batch(BATCH_SIZE) \
                          .prefetch(buffer_size = BUFFER_SIZE) \

## Setup Model

Clear keras session to clear memory usage from previous models.

In [34]:
tf.keras.backend.clear_session()

Create the model with a Dense Layer with `num_classes` nodes.

In [35]:
num_classes = len(set(df[TARGET_LABEL].values))

model = Model(num_classes)

Model: "custom_mobilenet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 128, 128, 3)]     0         
                                                                 
 random_flip (RandomFlip)    (None, 128, 128, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 128, 128, 3)      0         
 tion)                                                           
                                                                 
 random_contrast (RandomCont  (None, 128, 128, 3)      0         
 rast)                                                           
                                                                 
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 1280)         

Check if Google Colab is using the GPU.

In [36]:
if tf.test.gpu_device_name() == '/device:GPU:0':
  tf.device('/device:GPU:0')
  print("Using available GPU...")
else:
  print("No GPU available...")

Using available GPU...


Define Optimizer, Loss Functions and Metrics based on classification type.

In [37]:
# optimizer = tf.keras.optimizers.SGD(learning_rate = 0.02, momentum = 0.9, nesterov = True)
optimizer = tf.keras.optimizers.Adam()

# loss = tf.keras.losses.SparseCategoricalCrossentropy()
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.01)

# metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name = 'accuracy'),
#            tf.keras.metrics.SparseTopKCategoricalAccuracy(k = 5, name = 'top_5_accuracy'),
#            tf.keras.metrics.SparseTopKCategoricalAccuracy(k = 10, name = 'top_10_accuracy')]
metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'accuracy'),
           tf.keras.metrics.TopKCategoricalAccuracy(k = 5, name = 'top_5_accuracy'),
           tf.keras.metrics.TopKCategoricalAccuracy(k = 10, name = 'top_10_accuracy')]

In [38]:
model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

Create callback to automatically save model weights if there is an improvement. 

In [39]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = f"{checkpoint_dir}/{checkpoint_path}", save_weights_only = True, save_best_only = True, verbose = 1)

Create callback to automatically save logs for TensorBoard. 

In [40]:
log_callback = tf.keras.callbacks.TensorBoard(log_dir = logs_dir, write_graph = True, update_freq = 'epoch')

Create callback to automatically stop if validation loss doesn't improve significantly.

In [41]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10, min_delta = 0.01)

## Train Model

In [ ]:
history = model.fit(
  train_ds,
  epochs = EPOCHS,
  validation_data = val_ds,
  callbacks = [cp_callback, log_callback, es_callback],
)

model.evaluate(test_ds)

Epoch 1/50
534/534 [==============================] - ETA: 0s - loss: 9.3081 - accuracy: 0.0024 - top_5_accuracy: 0.0090
Epoch 00001: saving model to /content/drive/MyDrive/Project/checkpoints/checkpoint_mobilenetv2.ckpt
534/534 [==============================] - 711s 1s/step - loss: 9.3081 - accuracy: 0.0024 - top_5_accuracy: 0.0090 - val_loss: 9.9404 - val_accuracy: 0.0018 - val_top_5_accuracy: 0.0073
Epoch 2/50
534/534 [==============================] - ETA: 0s - loss: 9.0317 - accuracy: 0.0105 - top_5_accuracy: 0.0316
Epoch 00002: saving model to /content/drive/MyDrive/Project/checkpoints/checkpoint_mobilenetv2.ckpt
534/534 [==============================] - 702s 1s/step - loss: 9.0317 - accuracy: 0.0105 - top_5_accuracy: 0.0316 - val_loss: 10.3597 - val_accuracy: 0.0039 - val_top_5_accuracy: 0.0140
Epoch 3/50
534/534 [==============================] - ETA: 0s - loss: 8.3451 - accuracy: 0.0287 - top_5_accuracy: 0.0761
Epoch 00003: saving model to /content/drive/MyDrive/Project/chec

Save model.

In [ ]:
model.save_weights(f'{models_dir}/model_{model_name}_{model_version}.h5', overwrite = True)

In [ ]:
tf.keras.models.save_model(model, f'{models_dir}/model_{model_name}_{model_version}', save_format = 'h5', overwrite = True)

## Continue Model Training

In [42]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

model.load_weights(latest)

history = model.fit(
  train_ds,
  epochs = 12,
  validation_data = val_ds,
  callbacks = [cp_callback, log_callback, es_callback],
)

model.evaluate(test_ds)

Epoch 1/12
534/534 [==============================] - ETA: 0s - loss: 4.1805 - accuracy: 0.6950 - top_5_accuracy: 0.8872 - top_10_accuracy: 0.9315
Epoch 00001: val_loss improved from inf to 9.51793, saving model to /content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/checkpoints/checkpoint_mobilenetv2.ckpt
534/534 [==============================] - 678s 1s/step - loss: 4.1805 - accuracy: 0.6950 - top_5_accuracy: 0.8872 - top_10_accuracy: 0.9315 - val_loss: 9.5179 - val_accuracy: 0.1516 - val_top_5_accuracy: 0.2524 - val_top_10_accuracy: 0.3028
Epoch 2/12
534/534 [==============================] - ETA: 0s - loss: 4.0285 - accuracy: 0.6970 - top_5_accuracy: 0.8905 - top_10_accuracy: 0.9328
Epoch 00002: val_loss did not improve from 9.51793
534/534 [==============================] - 676s 1s/step - loss: 4.0285 - accuracy: 0.6970 - top_5_accuracy: 0.8905 - top_10_accuracy: 0.9328 - val_loss: 9.7553 - val_accuracy: 0.1223 - val_top_5_accuracy: 0.2167 - val_top_10_accuracy: 0.2641


[9.518389701843262,
 0.14659148454666138,
 0.2487954944372177,
 0.2967709004878998]

In [44]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

model.load_weights(latest)

model.save_weights(f'{models_dir}/model_{model_name}_{model_version}.h5', overwrite = True)

model.evaluate(test_ds)

77/77 [==============================] - 35s 446ms/step - loss: 9.3825 - accuracy: 0.1489 - top_5_accuracy: 0.2442 - top_10_accuracy: 0.2915


[9.38249683380127,
 0.1489492505788803,
 0.24418246746063232,
 0.29154279828071594]

Save model.

In [ ]:
model.save_weights(f'{models_dir}/model_{model_name}_{model_version}.h5', overwrite = True)

In [ ]:
tf.keras.models.save_model(model, f'{models_dir}/model_{model_name}_{model_version}', save_format = 'h5', overwrite = True)

## Statistics

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

View logs through TensorBoard.

In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/Colab Notebooks/Computer Vision/Project/logs"